# 可选：数据并行
这个教程我们将会介绍如何用多个 GPUs 来进行 <font color=red>DataParallel</font>。

Pytorch框架使得 GPUs 使用非常简单。我们可以把一个模型移到 GPU 上：

In [1]:
device = torch.device("cuda:0")
model.to(device)

NameError: name 'torch' is not defined

然后，你可以将所有的 tensors 复制到 GPU 上：

In [2]:
mytensor = my_tensor.to(device)

NameError: name 'my_tensor' is not defined

注意：只调用 <font color=red>my_tensor.to(device)</font> 返回的是 <font color=red>my_tensor</font> 的副本，而不是直接将 <font color=red>my_tensor</font> 覆盖掉。所以你需要将其赋值给一个新的 tensor 然后将这个新 tensor 用于 GPU 上。

在多个 GPUs 上执行前向（forward）和反向传播（backward propagation）是很自然的事。然而 Pytorch 默认只用一个 GPU。 你只要用 <font color=red>DataParallel(device)</font> 就可以将你的操作用多 GPUs 来执行。

In [ ]:
model = nn.DataParallel(model)

这是这节教程的核心。接下来，我们将讨论更多的细节。
## 导入和参数（import and parameters)
导入 Pytorch 模块以及定义参数。

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Parameters and DataLoaders
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

**设备**

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 假数据集（Dummy DataSet）
制作一个假的（随机的）数据集。你只需要实现下 \__getitem\__ 方法：

In [5]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size), 
                         batch_size=batch_size, shuffle=True)

## 简单模型
作为一个简单的demo，我们的模型只是接受输入，经过现行变化得到输出。当然，你可以在任何模型（如CNN，RNN，Capsule Net等）上用 <font color=red>DataParallel</font>。

我们已经在模型之中写上了print语句，用来监督输入输出 tensors 的大小（size）。请注意 batch rank0 处的打印信息。

In [3]:
class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

## 创建模型和数据并行
这是这节教程的核心。首先，我们需要创建一个模型的实例并且确认下我们是否拥有多个 GPUs。如果我们有多个 GPUs，我们可以用 <font colot=red>nn.DataParallel</font> 来封装我们的模型。之后，我们可以用 <font colot=red>model.to(device)</font> 把我们的模型转换到 GPUs 上计算。

In [4]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

NameError: name 'device' is not defined

## 运行模型
现在，我们可以看一下输入（input）和输出（output）的尺寸大小（size）：

In [5]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

NameError: name 'rand_loader' is not defined

## 结果
如果你没有 GPU 或者只有一个 GPU，那么当模型读入30 batch 的输入以及输出时，模型应该得到30个输出。但是如果你有多个 GPUs 的话，那么你将得到类似如下的结果。
### 2 GPUs
如果你有2个，你将看到：
```python
# on 2 GPUs
Let's use 2 GPUs!
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
    In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
    In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```
### 3 GPUs
如果你有3个，你将看到：
```python
Let's use 3 GPUs!
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
    In Model: input size torch.Size([10, 5]) output size torch.Size([10, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```
### 8 GPUs
如果你有8个，你将看到：
```python
Let's use 8 GPUs!
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([4, 5]) output size torch.Size([4, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
    In Model: input size torch.Size([2, 5]) output size torch.Size([2, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
```
## 总结
DataParalle 自动将数据分割并且把计算移到在不同 GPUs 上的多个模型上。当每个模型都结束他们的工作后，DataParallel 收集并合并这些计算结果，然后将合并后的结果返回给你。
更多的信息请查看：
http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 。